In [36]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2
import random
import os
import sys

# O 'ESTRAGADOR' DE IMAGENS 1001

In [37]:
def addReflection(img):

    # cria uma máscara com os reflexos em formato de círculo
    mask_reflections = np.zeros_like(img, dtype=np.uint8)

    for n in range(3):
        x = random.randint(0, 240)
        y = random.randint(0, 240)

        radius = random.randint(20, 150)

        cv2.circle(mask_reflections, (x, y), radius, (255, 255, 255), -1)

    # ajusta a intensidade dos pixels na área dos reflexos
    brightness_factor = 0.1 

    # multiplica a área dos reflexos pelo fator de brilho
    reflections = cv2.multiply(mask_reflections, brightness_factor)

    # sobrepõe os reflexos na imagem original
    reflection_img = cv2.add(img, reflections)

    return reflection_img

In [38]:
def adjustImage(img):

    epsilon = 1e-10 # numero bem pequeno para incluir o limite final no intervalo
    alpha = round(random.uniform(0.1, (1.5 + epsilon)), 2) # 1 mantém
    beta = random.randint(-5, 5) # 0 mantém

    # alpha ajusta o contraste e beta ajusta o brilho
    adjusted_image = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    cooked_img = addReflection(adjusted_image)

    return cooked_img

In [39]:
def noiseMaker(img):

    epsilon = 1e-10 # numero bem pequeno para incluir o limite final no intervalo
    std_dev = round(random.uniform(0.1, (0.6 + epsilon)), 2)
    mean = 0
    
    # ruído gaussiano
    noise = np.random.normal(mean, std_dev, img.shape).astype(np.uint8)
    noisy_img = cv2.add(img, noise)

    # transforma ruídos da imagem em escala de cinza
    #noisy_img = cv2.cvtColor(noisy_img, cv2.COLOR_BGR2GRAY)

    cooked_img = adjustImage(noisy_img)

    return cooked_img

In [40]:
def addDepth(img):
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))

    edges = cv2.Canny(img, 100, 200)
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)

    dilated_edges = dilated_edges.astype(np.float32) / 255

    # dilata o caracter
    dilated_edges[img > 0] = 0.25

    # altera as cores para o padrão de placa de carro
    depth_img = np.copy(dilated_edges)
    depth_img[dilated_edges == 0] = 0.65
    depth_img[dilated_edges == 1] = 0.25
    depth_img[dilated_edges == 0.25] = 0

    # borra a imagem para diminuir serrilhados
    depth_img = cv2.GaussianBlur(depth_img, (0,0), 0.93)

    depth_img = cv2.normalize(depth_img, None, 0, 255, cv2.NORM_MINMAX)
    depth_img = depth_img.astype(np.uint8)

    final_img = noiseMaker(depth_img)

    return final_img

# GERADOR DE CARACTERES

In [55]:
def singleDigitGenerator(character, counter, directory, chr_type):

    width = height = 240

    font = ImageFont.truetype("MANDATOR.ttf", size=180)

    img = Image.new('RGB', (width, height), color='black')
    imgDraw = ImageDraw.Draw(img)

    xText = width - random.randint(110, 370)
    yText = height - random.randint(110, 280)

    _, _, textWidth, textHeight = imgDraw.textbbox((xText,yText), character, font=font)
    xText = (width - textWidth) / 2
    yText = (height - textHeight) / 2

    imgDraw.text((xText, yText), character, font=font, fill='white')

    img.save(os.path.join(directory, 'chr%s_%d.png'% (character, counter)))

    img_path = f'./generatedCharacters/' + chr_type + f'/character_{character}/chr{character}_{counter}.png'
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    generated_img = addDepth(img)

    generated_img = Image.fromarray(generated_img)
    generated_img.save(os.path.join(directory, 'chr%s_%d.png'% (character, counter)))

# NÚMEROS

In [60]:
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

for num in numbers:

    directory = './generatedCharacters/numbers/character_%s'% num
    if not os.path.exists(directory): os.makedirs(directory)

    counter = 0
    
    for num_images in range(1300):
        singleDigitGenerator(num, counter, directory, chr_type='numbers')
        counter += 1

# LETRAS DO ALFABETO

In [61]:
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I',
            'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
            'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

for letter in alphabet:

    directory = './generatedCharacters/letters/character_%s'% letter
    if not os.path.exists(directory): os.makedirs(directory)

    counter = 0
    
    for num_images in range(1300):
        singleDigitGenerator(letter, counter, directory, chr_type='letters')
        counter += 1